# Creates single file csv from multiple csv files by year for KCPD Crime Data

###created: 6-10-2018 by: j.v.###
###updated: 6-13-2018 by: j.v. removed duplicated 2011 data when stacking csv's###

    

In [0]:
import pandas as pd

In [0]:
# Google drive connection using pydrive
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def gdrive_download_files(folder_id):
  """
  Purpose: Downloads all files from a folder on google drive which is specified by folder_id parameter.
  
  Parameters:
    folder_id: folder id for gdrive folder you wish to download files from     
   
  """ 
  
  dict = {}
  
  file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
  for file in file_list:
    print('title: %s' % (file['title']))   
    tmp_file_connection = drive.CreateFile({'id': '{}'.format(file['id'])})
    tmp_file_connection.GetContentFile(file['title'])
    dict[file['title']] = pd.read_csv(file['title'])
    
  return dict    
    
    

  

In [5]:
folder_id = '<FOLDER ID HERE>' #Hou's shared drive folder currently

# Calling above function to load all csv files into a dict
files_dict = gdrive_download_files(folder_id)



title: KCPD_Crime_Data_2017.csv
title: KCPD_Crime_Data_2009_Final.csv
title: KCPD_Crime_Data_2012_Final.csv
title: KCPD_Crime_Data_2011_Final.csv
title: KCPD_Crime_Data_2010_Final.csv
title: KCPD_Crime_Data_2013_Final.csv
title: KCPD_Crime_Data_2016.csv
title: KCPD_Crime_Data_2015.csv
title: KCPD_Crime_Data_2014.csv


In [0]:
import math, re

def get_lat_long_from_address(address):
  """
    Purpose: Splits address string on '\n' and retrives latitude and longitude and cast them to floats
    Parameters: address is a string which includes the address and lat,long coordinates delimited by '\n'
  """
  # Split out longitude and latitude from address string result_example => '(39.06740375200047, -94.53796609099965)'  
  
  try:     
    address_tuple = address.split('\n')    
    
    coordinate_pair_str = address_tuple[-1]    
    lat,long = map(float, re.sub("\(|\)", "" ,coordinate_pair_str).split(','))
    
    return (lat,long)
  except:   
    return (0.0,0.0)

In [7]:

# Data Preprocessing before concatinating datasets

"""
-- Some columns names had spaces vs '_' on others, so I grabbed the column names
   from the 2017 dataset and used that as a standard to process the other columns.
   
-- Years 2015 and 2016 had Latitude and Longitude columns, which I thought was usefull,
   so I derived these columns from the 'Location' column and added it to each year's
   dataframe.
-- Some years had named a column 'Location' and others named it 'Location 1', the prior
   was selected as the standard.
-- 'Firearm Used Flag' in some years had extra whitespace at the end of the string, this
    was removed.
-- Columns for all dataframes were aligned according to the 2017 data column order.

-- The data for the year 2013 didn't not have a Address or City column like all the other
   dataframes, so these were derived from the 'Location' columns.
"""



for key, df in files_dict.items():      
  lat_arr, long_arr = [], []
  
  print("Processing: ",key)
  
  # Remove extra whitespace from ends of column name ex => 'Firearm Used Flag  '
  df.rename(columns=lambda x: x.strip(), inplace=True)
  
  columns = ['Report_No', 'Reported_Date', 'Reported_Time', 'From_Date', 'From_Time',
       'To_Date', 'To_Time', 'Offense', 'IBRS', 'Description', 'Beat',
       'Address', 'City', 'Zip Code', 'Rep_Dist', 'Area', 'DVFlag', 'Invl_No',
       'Involvement', 'Race', 'Sex', 'Age', 'Firearm Used Flag', 'Latitude',
       'Longitude', 'Location']

  if 'Location 1' in df.keys():
    df.rename(columns={'Location 1':'Location'}, inplace=True)
      
  if 'Reported Time' in df.keys():
    df.rename(columns={'Reported Time':'Reported_Time'}, inplace=True)
    
  if 'From Time' in df.keys():
    df.rename(columns={'From Time':'From_Time'}, inplace=True) 
    
  if 'To Time' in df.keys():
    df.rename(columns={'To Time':'To_Time'}, inplace=True)   
    
    
  # Odd case 2013 data is the only one that doesn't have address or city columns
  # These can be created from the Location column
  
  if key == 'KCPD_Crime_Data_2013_Final.csv':
    df['Address'] = df['Location'].apply(lambda x: x.split('\n')[0:1])
    df['City'] = df['Location'].apply(lambda x: x.split('\n')[1:2][0].split(',')[0].strip())   
  
  
  for _, row in files_dict[key].iterrows():
    lat, long = get_lat_long_from_address(row['Location'])
    lat_arr.append(lat)
    long_arr.append(long)

  files_dict[key]['Latitude'] = lat_arr
  files_dict[key]['Longitude'] = long_arr
  
  # Aligns the columns
  files_dict[key] = files_dict[key][columns]

    
    


Processing:  KCPD_Crime_Data_2017.csv
Processing:  KCPD_Crime_Data_2009_Final.csv
Processing:  KCPD_Crime_Data_2012_Final.csv
Processing:  KCPD_Crime_Data_2011_Final.csv
Processing:  KCPD_Crime_Data_2010_Final.csv
Processing:  KCPD_Crime_Data_2013_Final.csv
Processing:  KCPD_Crime_Data_2016.csv
Processing:  KCPD_Crime_Data_2015.csv
Processing:  KCPD_Crime_Data_2014.csv


In [0]:
# Concatinate all processed year's dataframes into one file by vertical stacking
# the already aligned columns.

result = pd.concat(
    [
        files_dict['KCPD_Crime_Data_2009_Final.csv'], 
        files_dict['KCPD_Crime_Data_2010_Final.csv'],
        files_dict['KCPD_Crime_Data_2011_Final.csv'],        
        files_dict['KCPD_Crime_Data_2012_Final.csv'],
        files_dict['KCPD_Crime_Data_2013_Final.csv'],
        files_dict['KCPD_Crime_Data_2014.csv'],
        files_dict['KCPD_Crime_Data_2015.csv'],
        files_dict['KCPD_Crime_Data_2016.csv'],
        files_dict['KCPD_Crime_Data_2017.csv'],
        
    ])

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1160598 entries, 0 to 132138
Data columns (total 26 columns):
Report_No            1160598 non-null int64
Reported_Date        1160598 non-null object
Reported_Time        1160598 non-null object
From_Date            1158879 non-null object
From_Time            1158015 non-null object
To_Date              451534 non-null object
To_Time              475236 non-null object
Offense              1160598 non-null int64
IBRS                 1150867 non-null object
Description          1160598 non-null object
Beat                 1158023 non-null object
Address              1160456 non-null object
City                 1160442 non-null object
Zip Code             1137705 non-null float64
Rep_Dist             1156640 non-null object
Area                 1156613 non-null object
DVFlag               1160598 non-null object
Invl_No              1160598 non-null int64
Involvement          1160598 non-null object
Race                 1006582 non-null

In [0]:
# Convert concatenated dataframe to csv file
result.to_csv('kcpd_crime_2009-2017_concat.csv',index=False)

In [11]:
# Upload final dataframe to your personal drive folder
uploaded = drive.CreateFile({'title': 'kcpd_crime_2009-2017_concat.csv'})
uploaded.SetContentFile('kcpd_crime_2009-2017_concat.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1L-ey-hY0c5bRptP0Fxv-emD29JoA7hKu
